<a href="https://colab.research.google.com/github/zeynep68/CIFAR100/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [567]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.nn import Sequential, Conv2d, BatchNorm2d, ReLU, MaxPool2d, AvgPool2d, Softmax, Flatten
from torchvision import transforms
from torch.utils.data import DataLoader, sampler
from torchvision.datasets import CIFAR100 
from torch.optim.lr_scheduler import MultiStepLR

# Load and prepare data

In [568]:
train_mean = torch.Tensor([0.5072, 0.4867, 0.4411])
train_std = torch.Tensor([0.2668, 0.2559, 0.2756])
test_mean = torch.Tensor([0.5142, 0.4928, 0.4467])
test_std = torch.Tensor([0.2707, 0.2597, 0.2794])

In [569]:
BATCH_SIZE = 128
transform_train = transforms.Compose([transforms.ToTensor(), transforms.Normalize(train_mean, train_std)])
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(test_mean, test_std)])

In [570]:
train = CIFAR100(root='./trainset', train=True, download=True, transform=transform_train)
test = CIFAR100(root='./testset', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [571]:
len(train), len(test)

(50000, 10000)

In [572]:
NUM_TRAIN = 49000
train_loader = DataLoader(train, batch_size=BATCH_SIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN))) # 391
val_loader = DataLoader(train, batch_size=BATCH_SIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

In [580]:
class ConvolutionalModule(nn.Module):
  def __init__(self, input_channel=64, output_channel=128, stride=1, pad=0):
    super(ConvolutionalModule, self).__init__()
    self.layer1 = Sequential(
                    Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=1, stride=stride, padding=pad),
                    BatchNorm2d(output_channel),
                    ReLU()
                  )

    self.x_shortcut = Sequential(
                        Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=1, stride=stride, padding=0),
                        BatchNorm2d(output_channel),
                        ReLU()
                      )      

  def forward(self, x):
    return self.x_shortcut(x) + self.layer1(x)

In [581]:
class ReduceModule(nn.Module):
  def __init__(self, input_channel=64, output_channel=128):
    super(ReduceModule, self).__init__()
    self.layer1 = Sequential(
                    Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=1, stride=1, padding=0),
                    BatchNorm2d(output_channel),
                    ReLU()
                  )       
    
  def forward(self, x):
    return self.layer1(x)

In [582]:
class BasicModule(nn.Module):
  def __init__(self, num_channel):
    super(BasicModule, self).__init__()
    self.layer1 = Sequential(
                    Conv2d(in_channels=num_channel, out_channels=num_channel, kernel_size=1, stride=1, padding=0),
                    BatchNorm2d(num_channel),
                    ReLU()
                  )       
    self.layer2 = Sequential(
                    Conv2d(in_channels=num_channel, out_channels=num_channel, kernel_size=3, stride=1, padding=1),
                    BatchNorm2d(num_channel),
                    ReLU()
                  )

  def forward(self, x):
    return self.layer2(self.layer1(x))

In [583]:
class Bottleneck(nn.Module):
  def __init__(self, num_channel, stride=1, pad=0):
    super(Bottleneck, self).__init__()
    self.layer1 = Sequential(
                    BasicModule(num_channel[1]) if num_channel[0] == num_channel[1] else ReduceModule(num_channel[0], num_channel[1])
                  )
    self.layer2 = Sequential(
                    ConvolutionalModule(num_channel[1], num_channel[2], stride=stride, pad=pad)
                  )
    
    self.x_shortcut = Sequential(
                        Conv2d(in_channels=num_channel[0], out_channels=num_channel[2], kernel_size=1, stride=1, padding=0),
                        BatchNorm2d(num_channel[2]),
                        ReLU()
                      )

  def forward(self, x): 
    a = self.layer1(x)
    print(a.shape)
    b = self.layer2(a)
    print(b.shape)
    print()
    print()
    c = self.x_shortcut(x) + b
    print("foo")
    return c

In [584]:
class ResNet50(nn.Module):
  def __init__(self):
    super(ResNet50, self).__init__()
    self.conv1 = Sequential(
                         Conv2d(in_channels=3, out_channels=64, kernel_size=7 ,stride=2, padding=0),
                         BatchNorm2d(64),
                         ReLU(),
                         MaxPool2d(kernel_size=3, stride=2)
                       )
    
    self.conv2 = Sequential(
                   Bottleneck([64,64,256]),
                   Bottleneck([256,64,256]),
                   Bottleneck([256,64,256], stride=2, pad=1)
                 )
    
    self.conv3 = Sequential(
                   Bottleneck([256,128,512]),
                   Bottleneck([512,128,512]),
                   Bottleneck([512,128,512]),
                   Bottleneck([512,128,512])
                 )

    self.conv4 = Sequential(
                   Bottleneck([512,256,1024]),
                   Bottleneck([1024,256,1024]),
                   Bottleneck([1024,256,1024]),
                   Bottleneck([1024,256,1024]),
                   Bottleneck([1024,256,1024]),
                   Bottleneck([1024,256,1024])
                 )
    
    self.conv5 = Sequential(
                   Bottleneck([1024,512,2048]),
                   Bottleneck([2048,512,2048]),
                   Bottleneck([2048,512,2048])
                 )

    self.output_layer = Sequential(
                          AvgPool2d(kernel_size=1),
                          Flatten(),
                          Softmax(dim=1)
                        )
    

  def forward(self, x):
    y = self.conv1(x)
    y = self.conv2(y)
    #y = self.conv3(y)
    #y = self.conv4(y)
    #y = self.conv5(y)
    y = self.output_layer(y)
    return y

In [585]:
model = ResNet50()
model = model.to(device=DEVICE)
model.train() 
a = torch.randn([12,3,256,256])
a = a.to(device=DEVICE)
scores = model(a)

RuntimeError: ignored

# Function to check accuracy

In [ ]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('using device:', DEVICE)

In [ ]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on validation set')
  else:
    print('Checking accuracy on test set') 
  model.eval() 

  num_correct = 0
  num_samples = 0

  with torch.no_grad():
    for images, labels in loader: # durchläuft 391 mal (mit jeweils 128 Bildern)
      images = images.to(device=DEVICE)
      labels = labels.to(device=DEVICE)
      scores = model(images)
      _, predictions = scores.max(1)
      num_correct += (predictions == labels).sum()
      num_samples += predictions.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

# Training

In [ ]:
print_every = 100

In [ ]:
def train(model, optimizer, epochs=120):

  model = model.to(device=DEVICE)
  for epoch in range(epochs):
    scheduler.step()
    print('Epoch:', epoch,'LR:', scheduler.get_lr())
    for t, (images, labels) in enumerate(train_loader): # durchläuft 391 mal (mit jeweils 128 Bildern)
      model.train() 
      images = images.to(device=DEVICE)
      labels = labels.to(device=DEVICE)
      scores = model(images)
      loss = F.cross_entropy(scores, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if t % print_every == 0:
        print('Iteration %d, loss = %.4f' % (t, loss.item()))
        check_accuracy(val_loader, model)
        print()

In [ ]:
learning_rate = 1e-3
model = ResNet50()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = MultiStepLR(optimizer, milestones=[40,80, 100], gamma=0.1)

In [ ]:
train(model, optimizer)